In [18]:
import torch
import torch.nn as nn
from d2l import torch as d2l

### 图像卷积

In [2]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0]-h+1, X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h, j:j+w]*K).sum()
    return Y

In [3]:
corr2d(torch.arange(9).reshape(3,3), torch.tensor([[0., 1.],[2.,3.]]))

tensor([[19., 25.],
        [37., 43.]])

In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [5]:
X = torch.ones((6, 8))
X[:,2:6] = 0
K = torch.tensor([[1., -1.]])
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [6]:
corr2d(X.T, K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [7]:
conv2d = nn.Conv2d(1,1, kernel_size=(1,2), bias=False)

X = X.reshape(1, 1, 6, 8)
Y = Y.reshape(1,1,6,7)
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y-Y_hat)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    print(f'epoch {i+1}, loss: {l.sum()}')
print(conv2d.weight.data.reshape(1,2))

epoch 1, loss: 25.763151168823242
epoch 2, loss: 12.195411682128906
epoch 3, loss: 6.046647071838379
epoch 4, loss: 3.1496074199676514
epoch 5, loss: 1.7207353115081787
epoch 6, loss: 0.9804331064224243
epoch 7, loss: 0.5779826045036316
epoch 8, loss: 0.34963569045066833
epoch 9, loss: 0.21546298265457153
epoch 10, loss: 0.1344950795173645
tensor([[ 1.0226, -0.9486]])


### 填充和步幅

In [8]:
def padding2d(X, ph, pw):
    h, w = X.shape
    Y = torch.zeros((h+2*ph, w+2*pw))
    Y[ph:ph+h, pw:pw+w] = X
    return Y

In [9]:
X = torch.arange(9).reshape(3,3)
padding2d(X, 1, 0)

tensor([[0., 0., 0.],
        [0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.],
        [0., 0., 0.]])

### 多输入多通道

In [10]:
def corr2d_multi_in(X, K):
    return sum(corr2d(x, k) for x, k in zip(X, K))

In [11]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
print(X,'\n',K)
corr2d_multi_in(X,K)

tensor([[[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]]) 
 tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]])


tensor([[ 56.,  72.],
        [104., 120.]])

In [12]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [13]:
K = torch.stack([K, K+1, K+2], 0)
K.shape

torch.Size([3, 2, 2, 2])

In [14]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [15]:
def corr2d_multi_in_out_1x1(X, K):
    '''
    使用全连接层实现 1x1 卷积
    '''
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape(c_i, h*w)
    K = K.reshape(c_o, c_i)
    Y = torch.matmul(K, X)
    return Y.reshape(c_o, h, w)

In [16]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

Y1-Y2

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])

### 汇聚层(池化层)

In [23]:
def pool2d(X, pool_size, mode='max'):
    h, w = X.shape
    p_h, p_w = pool_size
    Y = torch.zeros((h-p_h+1, w-p_w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j] = X[i:i+p_h, j:j+p_w].max()
            if mode == 'avg':
                Y[i,j] = X[i:i+p_h, j:j+p_w].float().mean()
    return Y

In [21]:
X = torch.arange(9).reshape((3,3))
pool2d(X, (2,2), 'max')

tensor([[4., 5.],
        [7., 8.]])

In [24]:
pool2d(X, (2,2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

In [38]:
X = torch.arange(9).reshape((1,1,3,3)).float()
X = torch.cat([X, X+1], 1)
X

tensor([[[[0., 1., 2.],
          [3., 4., 5.],
          [6., 7., 8.]],

         [[1., 2., 3.],
          [4., 5., 6.],
          [7., 8., 9.]]]])

In [37]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[8.]],

         [[9.]]]])